# Preparing data


1) Building the London tubestations dataset

In [58]:
import pandas as pd # library for data analysis
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML documents
import re

In [59]:
# get the response in the form of html We send a GET request to the Wikipedia URL whose table needs to be scraped and store the HTML response in a variable. It is not legal to scrape any website, so we check the status code. 200 shows that you can go ahead and download it.
wikiurl="https://en.wikipedia.org/wiki/List_of_London_railway_stations"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)
print(response.status_code)

# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(response.text, 'html.parser')
tubetable=soup.find('table',{'class':"wikitable"})

df=pd.read_html(str(tubetable))
# convert list to dataframe
df=pd.DataFrame(df[0])

200


In [60]:
# Cleanup the 'Station' field. There are some numbers in square brackets in some of the station names which could cause issues when we try to use the station names

pattern = '([[0-9]+])'
df = df.replace(to_replace = pattern, value = '', regex = True)

#optional 
#suffix the station names by 'station, London, UK' so that we can add the lat long easily using Google maps API. Not doing this can cause issues, as some of the stations names are found in many other countries such as canada, newzealand etc.
df['Station'] = df['Station'].astype(str) +' station, London, UK'
df['Station']

#The coordinates column is messy and contains the coordingates in minutes and seconds as well as decimal formats 
#51°35′11″N 0°00′08″W﻿ / ﻿51.5864°N 0.0021°W
#split the coordinates column, split and cleanup latitude and longitude columns

df[['tobedropped', 'Latlong']] = df['Coordinates'].str.split(' / ', 1, expand=True)
df=df.drop(['tobedropped', 'Coordinates'], axis= 1)
df[['latitude', 'longitude']] = df['Latlong'].str.split('N', 1, expand=True)
df = df.drop('Latlong', axis=1)

#remove all the non numeric characters and symbols
df['latitude']=df['latitude'].str.replace("°", "")
df['longitude']=df['longitude'].str.replace("°E", "")
df['longitude']=df['longitude'].str.replace("°W", "")

#It also contains some zero break no-break space codepoints denoted by '/ufeff'
#lets only retain the decimal format lat long values and convert them into floats for further use.

df['latitude'] = df.latitude.str.replace("\ufeff", "")
df['latitude'] = df['latitude'].astype(float)
df['longitude'] = df.longitude.str.replace("\ufeff", "")
df['longitude'] = df['longitude'].astype(float)


In [61]:
df
#df.to_csv('london_stations_zones.csv')
#write df to csv file


,Station,Local authority,Managed by,Stationcode,Farezone,Yearopened,Category,latitude,longitude
0,"Abbey Wood station, London, UK",Greenwich,Elizabeth line,ABW,4,1849,C,51.491500,0.122900
1,"Acton Central station, London, UK",Ealing,London Overground,ACC,3,1853,D,51.508800,0.263400
2,"Acton Main Line station, London, UK",Ealing,Elizabeth line,AML,3,1868,E,51.516900,0.266900
3,"Albany Park station, London, UK",Bexley,Southeastern,AYP,5,1935,D,51.435800,0.126600
4,"Alexandra Palace station, London, UK",Haringey,Great Northern,AAP,3,1873,D,51.598300,0.119700
...,...,...,...,...,...,...,...,...,...
369,"Wood Street station, London, UK",Waltham Forest,London Overground,WST,4,1873,D,51.586400,0.002100
370,"Woolwich station, London, UK",Greenwich,Elizabeth line,WWC,2,2022,unlisted,51.491578,0.071819
371,"Woolwich Arsenal station, London, UK",Greenwich,Southeastern,WWA,4,1849,C,51.489800,0.069400
372,"Woolwich Dockyard station, London, UK",Greenwich,Southeastern,WWD,3,1849,E,51.491300,0.053600


2) Building the postcodes database

In [ ]:
# download the London postcodes csv file from https://www.doogal.co.uk/UKPostcodesCSV/?area=London
postcodes_csv_data = pd.read_csv("London_postcodes.csv")
# retain postcodes in use
postcodes = postcodes_csv_data[['Postcode','In Use?', 'Latitude', 'Longitude']]
postcodes_data = postcodes.loc[postcodes['In Use?'] == "Yes"]
# retain postcode, district, latitude and longitude columns
postcodes_latlong = postcodes [['Postcode','District', 'Latitude', 'Longitude']]

#save dataframe as csv 
postcodes_latlong.to_csv('london_postcodes_latlong.csv')

3) Appending nearest tubestation to all postcodes

In [ ]:
from geopy.distance import geodesic #to calculate distance between two lat-long coordinates
import numpy as np
import csv

In [ ]:
tubestations = pd.read_csv("london_stations_zones.csv")
tubestations.drop(['Local authority', 'Managed by', 'Yearopened', 'Category'],  axis=1, inplace=True)
tubestations['latitude'] = tubestations['latitude'].str.replace('_','')
tubestations["latlong"] = list(zip(tubestations.latitude.astype(float), tubestations.longitude.astype(float)))
tubestations_latlong = tubestations["latlong"]

postcodes = pd.read_csv('london_postcodes_latlong.csv')
postcodes["latlong"] = list(zip(postcodes.Latitude, postcodes.Longitude))

In [ ]:
#add the index of closest tubestation from london_postcodes_latlong.csv to an empty list  
# This way, the order of postcodes will be retained, and can be used later to reconstruct dataframe


closeststation = []
k = 0
for postcode in postcodes_latlong:
    k = k+1
    print("running postcode no:", k)
    i = -1
    dist = None
    for tubestation in tubestations_latlong:
        i = i+1
        
        if dist is None:
            dist = geodesic(postcode, tubestation).km
            #print("initiated for postcode", postcode)
        if dist <= geodesic(postcode, tubestation).km:
            continue
        else:
            dist = geodesic(postcode, tubestation).km
            j = i
    print(j)
    closeststation.append(j)
    
closeststations = pd.DataFrame(data = closeststation)
    


In [ ]:

#optional: since the above code processes 3,27,000 postcodes, it tends to run for a long time.
#it might be useful to save this dataframe as a csv to avoid having to run it repeatedly
#closeststations.to_csv("closeststations.csv")

In [ ]:
#fetch tubestation name and zone from tubestations dataframe and append to postcodes dataframe

tubenamelist=[]
tubezone = []
for station in closeststation:
    tubenamelist.append(tubestations.iloc[station]['Station'])
    tubezone.append(tubestations.iloc[station]['Farezone'])

postcodes['Nearest_tube_station'] = tubenamelist
postcodes['tube_zone'] = tubezone

#save as csv
postcodes.to_csv("postcodes_latlong_districts_neareststations.csv")

4) Cleaning tube fares data

In [62]:
#Tube fares data available as a pdf (downloaded from https://content.tfl.gov.uk/adult-fares.pdf) was copied to MS exccel 
# Monthly fare rates for adults w/o any special discount were saved as a csv file: adult_tube_fares.csv
tubefares = pd.read_csv("adult_tube_fares.csv") 

#perform some cleanup step, remove symbols and punctuations from monthly fare column, convert to numeric
tubefares['Monthly'] = tubefares['Monthly'].str.replace('£', '')
tubefares['Monthly'] = tubefares['Monthly'].str.replace(',', '')
tubefares['Monthly'] = pd.to_numeric(tubefares['Monthly'])

# cleanup other columns
tubefares.drop(['Day off-peak', 'Annual', '7 Day'], axis = 1, inplace=True)

tubefares['zones'] = tubefares['zones'].str.replace('Zones', '')
tubefares['zones'] = tubefares['zones'].str.replace('only', '')
tubefares['zones'] = tubefares['zones'].str.replace('Zone', '')

#save dataframe as csv, edit using spreadsheet software to produce all possiblities from zone and to zone eg. 2-1. 1-2 
tubefares.to_csv('zones_tubefares_cleaned.csv')
#the edited csv is named tubefares_by_zones.csv

,fare_category,zones,Monthly
0,1,1,147.5
1,2,1-2,147.5
2,3,1-3,173.6
3,4,1-4,212.0
4,5,1-5,252.3
5,6,1-6,270.0
6,7,1-7,293.8
7,8,1-8,346.8
8,9,1-9,384.8
9,10,2,110.6


5) Cleaning up rightmove rents data

In [ ]:
#load rent data csv created using rightmove_rent_data_scraper.ipynb
rentdata = pd.read_csv("rents_data_with_boroughs.csv") 

#cleanup Borough and price columns to correct artefacts and remove entries having no rent information
pat = '[\W\s+]'
rentdata['Borough'] = rentdata['Borough'].str.replace(pat, '')
rentdata['Borough'] = rentdata['Borough'].str.replace('CityofLondon', 'City of London')
rentdata['Price'] = rentdata['Price'].str.replace('£', '')
rentdata['Price'] = rentdata['Price'].str.replace(',', '')
rentdata['Price'] = rentdata['Price'].str.replace(' pcm', '')
rentdata = rentdata[rentdata['Price'].str.contains("POA")==False]

#convert rent values to numeric
rentdata['Price'] = pd.to_numeric(rentdata['Price'])

#generalize property descriptions and categorize into studio, 1bhk, 2bhk, 3bhk
cat1 = '^1 bedroom(.+)'
cat2 = '^2 bedroom(.+)'
cat3 = '^3 bedroom(.+)'
cat4 = '^Studio(.+)'
rentdata['Description'] = rentdata['Description'].str.replace(cat1, '1bhk')
rentdata['Description'] = rentdata['Description'].str.replace(cat2, '2bhk')
rentdata['Description'] = rentdata['Description'].str.replace(cat3, '3bhk')
rentdata['Description'] = rentdata['Description'].str.replace(cat4, 'studio')

In [ ]:
# create dataframes for types of property
rentdata_1bhk = rentdata.loc[rentdata['Description']=='1bhk']
rentdata_2bhk = rentdata.loc[rentdata['Description']=='2bhk']
rentdata_3bhk = rentdata.loc[rentdata['Description']=='3bhk']
rentdata_studio = rentdata.loc[rentdata['Description']=='studio']

In [ ]:
#summarize price data for different types of property by borough
pivot_studio = rentdata_studio.pivot_table(index=['Borough'], values=['Price'], aggfunc={'median','mean','min'})
pivot_1bhk = rentdata_1bhk.pivot_table(index=['Borough'], values=['Price'], aggfunc={'median','mean','min'})
pivot_2bhk = rentdata_2bhk.pivot_table(index=['Borough'], values=['Price'], aggfunc={'median','mean','min'})
pivot_3bhk = rentdata_3bhk.pivot_table(index=['Borough'], values=['Price'], aggfunc={'median','mean','min'})

#save pivot tables as csv 
pivot_1bhk.to_csv("pivot_1bhk.csv")
pivot_2bhk.to_csv("pivot_2bhk.csv")
pivot_3bhk.to_csv("pivot_3bhk.csv")
pivot_studio.to_csv("pivot_studio.csv")